<a href="https://colab.research.google.com/github/tmendonca28/DLNID/blob/master/SimpleMLP_3L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from statistics import mean
import pandas as pd
import numpy as np

In [0]:
# Loading the data
from google.colab import drive
drive.mount('/content/drive')

# drive.mount("/content/drive", force_remount=True)
train_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTrainRenameNominalValuesNormalized.csv", header=0, low_memory=False, dtype = np.float32)
test_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTestRenameNominalValuesNormalized.csv", header=0, low_memory=False, dtype = np.float32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
targets_pd = train_data.iloc[0:, 41]
features_pd = train_data.iloc[0:, 0:41]

test_targets_pd = test_data.iloc[0:, 41]
test_features_pd = test_data.iloc[0:, 0:41]

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.FloatTensor(features_pd.values)
# torch_tensor = torch.tensor(targets_df['targets'].values)
targetsTrain = torch.LongTensor(targets_pd.values)

featuresTest = torch.FloatTensor(test_features_pd.values)
# torch_tensor = torch.tensor(targets_df['targets'].values)
targetsTest = torch.LongTensor(test_targets_pd.values)

# batch_size, epoch and iteration
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(features_pd) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest, targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [0]:
# Building a feed-forward network
model = nn.Sequential(nn.Linear(41, 1000),
                     nn.ReLU(),
                     nn.Linear(1000, 64),
                     nn.ReLU(),
                     nn.Linear(64, 2),
                     nn.LogSoftmax(dim=1))

In [0]:
input_dim = 41    # input dimension
hidden_dim = 240  # hidden layer dimension; was 100 before
layer_dim =  1    # number of hidden layers; was 2 before
output_dim = 2   # output dimension

# Cross Entropy Loss 
criterion = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

seq_dim = 1  
loss_list = []
iteration_list = []
accuracy_list = []
acc = []
count = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # print("Training: ",i)
#         pdb.set_trace()
        train  = Variable(images.view(-1, 41))
        labels = Variable(labels )
        # pdb.set_trace()
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        # Calculate softmax and cross entropy loss
        loss = criterion(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 250 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images.view(-1, 41))
                
                # Forward propagation
                outputs = model(images)
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 500 == 0:
                # Print Loss
                acc.append(accuracy.item())
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data.item(), accuracy))

print("{0:.2f}".format(mean(acc)))

Iteration: 500  Loss: 0.2859121859073639  Accuracy: 80 %
Iteration: 1000  Loss: 0.22439804673194885  Accuracy: 76 %
Iteration: 1500  Loss: 0.16768355667591095  Accuracy: 76 %
Iteration: 2000  Loss: 0.19020555913448334  Accuracy: 80 %
Iteration: 2500  Loss: 0.12939220666885376  Accuracy: 78 %
Iteration: 3000  Loss: 0.0857904702425003  Accuracy: 77 %
Iteration: 3500  Loss: 0.17980743944644928  Accuracy: 78 %
Iteration: 4000  Loss: 0.15235891938209534  Accuracy: 78 %
Iteration: 4500  Loss: 0.15313546359539032  Accuracy: 78 %
Iteration: 5000  Loss: 0.0975704938173294  Accuracy: 75 %
Iteration: 5500  Loss: 0.0574466846883297  Accuracy: 75 %
Iteration: 6000  Loss: 0.20054876804351807  Accuracy: 79 %
Iteration: 6500  Loss: 0.07474350929260254  Accuracy: 74 %
Iteration: 7000  Loss: 0.08719532936811447  Accuracy: 77 %
Iteration: 7500  Loss: 0.11928638815879822  Accuracy: 74 %
Iteration: 8000  Loss: 0.06544480472803116  Accuracy: 78 %
Iteration: 8500  Loss: 0.10434421896934509  Accuracy: 78 %
77